In [106]:
import os
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.multitest import multipletests
from scipy.stats import ttest_ind_from_stats
import plotly.graph_objs as go
from plotly.offline import plot
import scipy
import json
import plotly.io as pio
from plotly.offline import iplot
import plotly as py
from matplotlib_venn import venn2, venn3
import matplotlib.pyplot as plt
import matplotlib
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.stats import t
import seaborn as sns
from datetime import datetime, timedelta
import plotly.express as px
from pathlib import Path
import scipy.stats
import math
from select import select
import numpy as np
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth

#only used for the app

# import django
# from django.conf import settings
# from django.contrib.auth.decorators import login_required, permission_required
# from file_manager.models import DataAnalysisQueue, SampleRecord, \
#     SavedVisualization, VisualizationApp, UserSettings, ProcessingApp
# from django.shortcuts import render
# from django.conf import settings


In [107]:
#constants 
Group_LIST_NAME = ["group1_record", "group2_record",
                   "group3_record", "group4_record",
                   "group5_record", "group6_record"]
GROUP_NAME = ["group1_name", "group2_name", "group3_name",
              "group4_name", "group5_name", "group6_name"]
saved_settings ={}
plot_options = {}
JUPYTER_MODE = "JPY_PARENT_PID" in os.environ #check if it's in jupiter notebook mode
APPFOLDER = "./"
url_base = None

#settings
WRITE_OUTPUT = False
USE_MaxLFQ = False


In [108]:
''' This is only for the webapp, not for jupyter notebook
def get_run_name(queue_id):
    """_Get the run name/sample list from the result files, only
    used for webapp for populate dropdown list_
    Args:
        queue_id (_int_): _task id from the process queue_
    Returns:
        _type_: _pandas data serial contains experiment list_
        0              sample1
        1              sample2
        2              sample3
    """
    if not queue_id:
        return None
    # get processing name
    process_app = DataAnalysisQueue.objects.filter(
        pk=queue_id).first().processing_app.name
    # fragpipe results
    if "FragPipe" in process_app:
        peptide_file = DataAnalysisQueue.objects.filter(
            pk=queue_id).first().output_file_2
        peptide = pd.read_table(peptide_file)
        #
        # get experiment names from columns names containning " MaxLFQ Intensity"
        run_metadata = [
            col for col in peptide.columns if " MaxLFQ Intensity" in col]
        # remove " MaxLFQ Intensity" from the experiment names
        run_metadata = [name.replace(" MaxLFQ Intensity", "")
                            for name in run_metadata]
        # create a pandas series to store the experiment names
        run_metadata = pd.Series(run_metadata)
    elif "PD" in process_app:
        inpufile_6 = DataAnalysisQueue.objects.filter(
            pk=queue_id).first().output_file_6
        meta_table = pd.read_table(inpufile_6)
        # Replace single backslashes with forward slashes in the 'file_paths' column
        meta_table['File Name'] = meta_table['File Name'].str.replace('\\', '/', regex=False)
        # Apply a lambda function to extract file names without extensions
        meta_table['file_names'] = meta_table['File Name'].apply(lambda x: os.path.splitext(os.path.basename(x))[0])
        run_metadata =meta_table['file_names']

    else:
        run_metadata = pd.Series()

    return run_metadata
'''

' This is only for the webapp, not for jupyter notebook\ndef get_run_name(queue_id):\n    """_Get the run name/sample list from the result files, only\n    used for webapp for populate dropdown list_\n    Args:\n        queue_id (_int_): _task id from the process queue_\n    Returns:\n        _type_: _pandas data serial contains experiment list_\n        0              sample1\n        1              sample2\n        2              sample3\n    """\n    if not queue_id:\n        return None\n    # get processing name\n    process_app = DataAnalysisQueue.objects.filter(\n        pk=queue_id).first().processing_app.name\n    # fragpipe results\n    if "FragPipe" in process_app:\n        peptide_file = DataAnalysisQueue.objects.filter(\n            pk=queue_id).first().output_file_2\n        peptide = pd.read_table(peptide_file)\n        #\n        # get experiment names from columns names containning " MaxLFQ Intensity"\n        run_metadata = [\n            col for col in peptide.column

In [109]:

def queue_info_api(queue_id, server_address, user_name, password):
    """_Get the queue and app info from the server through API, this
    is only for the jupyter notebook, not for the webapp_
    """

    authinfo = HTTPBasicAuth(user_name, password)

    #get queue info and test if the user name and password are correct   

    queue_response = requests.get(
        f'http://{server_address}/files/api/DataAnalysisQueue/{queue_id}/',
        auth=authinfo
    )
    if queue_response.status_code != 200:
        raise Exception("Invalid username or password")
    else:
        queue_json_data = queue_response.json()

    # Get app information
    app_response = requests.get(
        f"http://{server_address}/files/api/ProcessingApp/{queue_json_data['processing_app']}/",
        auth=authinfo
    )
    app_json_data = app_response.json()
    return queue_json_data, app_json_data


In [110]:
def read_files(queue_ids = None, queue_info = None, processor_info = None, grouped_input_files = []):
    '''
    Creates a list of data objects

    grouped_input_files
    {input1:
    input2:
    input3:
    input4:
    input5:
    process_app:
    }
    '''


    data_objects = []

    prefix_letters = ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
    i = 0
    for eachGroup in grouped_input_files:
        if queue_ids is not None:
            pass
        else:
            #print(eachGroup)
            process_app = eachGroup["process_app"]
            input1= eachGroup["input1"]
            input2= eachGroup["input2"]  
            input3= eachGroup["input3"]
            input4= eachGroup["input4"]  
            input5= eachGroup["input5"]

        current_data_object = read_file(input1=input1,input2=input2,
                                        input3=input3,input4 = input4,
                                        input5=input5, process_app=process_app,prefix_letter = prefix_letters[i])
        data_objects.append(current_data_object)
        
        i = i + 1

    
    return data_objects




In [111]:
def read_file(queue_id=None, queue_info= None, processor_info = None,
               input1=None, input2=None,input3=None, input4=None, input5=None,
            process_app = None, prefix_letter = "A"):
    """_Read data from data manager API or through local files or read directly
    in the webapp_
    Args:
        queue_id (_int_): _processing queue id_
        queue_info (_dict_): _queue info from the API_
        processor_info (_dict_): _processor info from the API_        
        input1 (_str_): _input file 1_
        input2 (_str_): _input file 2_
        input3 (_str_): _input file 3_
        input4 (_str_): _input file 4_
        input5 (_str_): _input file 5_
        process_app (_str_): _process app name_
    Returns:
        _dict_: _dictionary containing data all data        
    """

    min_unique_peptides = 1

    #getting files from data system
        # getting files from data system (webapp)
    if queue_id is not None and processor_info is None:
        # Method 1 pull data directly (used by the webapp)
        process_app = DataAnalysisQueue.objects.filter(
            pk=queue_id).first().processing_app.name
        input1= DataAnalysisQueue.objects.filter(
            pk=queue_id).first().output_file_1
        input2= DataAnalysisQueue.objects.filter(
            pk=queue_id).first().output_file_2  
        input3= DataAnalysisQueue.objects.filter(
            pk=queue_id).first().output_file_3
        input4= DataAnalysisQueue.objects.filter(
            pk=queue_id).first().output_file_4  
        input5= DataAnalysisQueue.objects.filter(
            pk=queue_id).first().output_file_5
    elif queue_info is not None and processor_info is not None:
    # Method 2 pull data from the data system (used by jupyter notebook)
        process_app = processor_info["name"]
        input1= queue_info["output_file_1"]
        input2= queue_info["output_file_2"]  
        input3= queue_info["output_file_3"]
        input4= queue_info["output_file_4"]  
        input5= queue_info["output_file_5"]
    # method 3 feed data directly (through local file paths)
    else:
        analysis_file = input1

    if "FragPipe" in process_app:     # fragpipe results
        # read data
        peptide_table = pd.read_table(input2,low_memory=False)
        protein_table = pd.read_table(input1,low_memory=False)

        # filter Contaminant
        peptide_table= peptide_table[~peptide_table[
            'Mapped Proteins'].str.contains(
            "contam_sp", na=False)]
        protein_table= protein_table[~protein_table['Protein'].str.contains(
            "contam_sp", na=False)].query(
            "`Combined Total Peptides` >= @min_unique_peptides")
        
        # get experiment names from columns names containning "Intensity"
        # or " MaxLFQ Intensity" if MaxLFQ is used

        if USE_MaxLFQ:
            column_tail = " MaxLFQ Intensity"
            intensity_cols = protein_table.columns[
                protein_table.columns.str.contains(column_tail)].tolist()
        else:
            column_tail = " Intensity"
            cols = protein_table.columns
            intensity_cols =  protein_table.columns[( \
                cols.str.contains(" Intensity")) & \
                (~cols.str.contains(" MaxLFQ Intensity"))]
        #get the column names of protein_table that contain "Intensity" but not " MaxLFQ"
        
        ## Proteins abundance table
        protein_table.rename(columns={'Protein ID': 'Accession'},inplace=True)

        all_path_cols = intensity_cols.append(pd.Index(['Accession']))

        prot_abundance = protein_table.loc[:, all_path_cols]


        ## Peptide abundance table
        peptide_table.rename(columns={'Peptide Sequence': 'Annotated Sequence'}, inplace=True)

        all_path_cols = intensity_cols.append(pd.Index(['Annotated Sequence']))



        pep_abundance = peptide_table.loc[:, all_path_cols]


        # remove " MaxLFQ Intensity" or " Intensity" from names
        run_name_list = [name.replace(column_tail, "")
                            for name in intensity_cols]
        
        run_name_list = pd.DataFrame({"Run Names": run_name_list})
        run_name_list['Run Identifier'] = run_name_list.index.to_series().apply(lambda x: prefix_letter + str(x))

        prot_abundance = prot_abundance.rename(columns={
            col: col.replace(column_tail, "") for col in
              prot_abundance.columns if column_tail in col})

        pep_abundance = pep_abundance.rename(columns={
            col: col.replace(column_tail, "") for col in
              pep_abundance.columns if column_tail in col})

        prot_abundance = prot_abundance.rename(dict(zip(run_name_list["Run Names"],run_name_list["Run Identifier"])))
        pep_abundance = pep_abundance.rename(dict(zip(run_name_list["Run Names"],run_name_list["Run Identifier"])))

        # get ID matrix tables
        prot_ID = prot_abundance.copy()
        cols = [col for col in prot_ID.columns if col != 'Accession']
        for col in cols:
            if prot_ID[col].dtype != 'object': # Check if not a string column
                prot_ID[col].replace(0, np.nan, inplace=True)
                # Replace all numerical values to ID
                prot_ID[col] = prot_ID[col].astype(str).str.replace("\d+\.\d+", "ID", regex=True)
        pep_ID = pep_abundance.copy()
        cols = [col for col in pep_ID.columns if col != 'Annotated Sequence	']
        for col in cols:
            if pep_ID[col].dtype != 'object': # Check if not a string column
                pep_ID[col].replace(0, np.nan, inplace=True)

                # Replace all numerical values to ID
                pep_ID[col] = pep_ID[col].astype(str).str.replace("\d+\.\d+", "ID", regex=True)
        prot_other_info = protein_table.loc[
            :, ~protein_table.columns.str.contains('Intensity')]
        prot_other_info["Source_File"] = input1
        pep_other_info = peptide_table.loc[
            :, ~peptide_table.columns.str.contains('Intensity')]
        pep_other_info["Source_File"] = input2


    elif "DIANN" in process_app:
        # read in DIANN output files
        peptide_table = pd.read_table(input2,low_memory=False)
        protein_table = pd.read_table(input1,low_memory=False)
        prot_other_info = pd.read_table(input3,low_memory=False)
        pep_other_info = pd.read_table(input4,low_memory=False)

        prot_other_info["Source_File"] = input3
        pep_other_info["Source_File"] = input4

        meta_table = pd.read_csv(input5, sep=' ', header=None, names=["File Name"])
        meta_table.reset_index(drop=True, inplace=True)
        # filter Contaminant
        protein_table= protein_table[~protein_table['Protein.Group'].str.contains(
            "contam_sp", na=False)]
        peptide_table= peptide_table[~peptide_table['Protein.Group'].str.contains(
            "contam_sp", na=False)]
        prot_other_info= prot_other_info[~prot_other_info['Protein'].str.contains(
            "contam_sp", na=False)]
        pep_other_info= pep_other_info[~pep_other_info['Mapped Proteins'].str.contains(
            "contam_sp", na=False)]
        
        prot_other_info.rename(columns={'Protein': 'Accession'}, inplace=True)
        pep_other_info.rename(columns={'Modified.Sequence': 'Annotated Sequence'}, inplace=True)
        # Replace backslashes with forward slashes if data comes from Windows
        meta_table['File Name'] = meta_table['File Name'].str.replace('\\', '/', regex=False)
        # Apply a lambda function to extract file names without extensions
        meta_table['File Name'] = meta_table['File Name'].apply(lambda x: os.path.splitext(os.path.basename(x))[0])
        run_name_list = meta_table['File Name'].tolist()
        run_name_list = pd.DataFrame({"Run Names": run_name_list})
        run_name_list['Run Identifier'] = run_name_list.index.to_series().apply(lambda x: prefix_letter + str(x))


        # Get the file names from the meta table
        protein_path_cols = protein_table.filter(regex='\\\\|Protein.Ids').columns

        ## Proteins
        prot_abundance = protein_table.loc[:, protein_path_cols]
        # Rename Columns to remove file path
        file_path_cols = protein_table.filter(regex='\\\\').columns
        prot_abundance.columns = [os.path.splitext(os.path.basename(x))[0] if x in file_path_cols else x for x in prot_abundance.columns]
        prot_abundance = prot_abundance.rename(columns={'Protein.Ids': 'Accession'})
        prot_abundance = prot_abundance.rename(columns = dict(zip(run_name_list["Run Names"],run_name_list["Run Identifier"])))
        #convert to str for IDs matrix
        prot_ID = prot_abundance.copy()
        cols = [col for col in prot_ID.columns if col != 'Accession']
        for col in cols:
            if prot_ID[col].dtype != 'object': # Check if not a string column
                prot_ID[col].replace(0, np.nan, inplace=True)
                # Replace all numerical values to ID
                prot_ID[col] = prot_ID[col].astype(str).str.replace("\d+\.\d+", "ID", regex=True)
        ## Peptides
        peptide_path_cols = peptide_table.filter(regex='\\\\|Modified.Sequence').columns

        pep_abundance = peptide_table.loc[:, peptide_path_cols]
        pep_abundance = pep_abundance.rename(dict(zip(run_name_list["Run Names"],run_name_list["Run Identifier"])))

        # Rename Columns to remove file path
        file_path_cols = peptide_table.filter(regex='\\\\').columns
        pep_abundance.columns = [os.path.splitext(os.path.basename(x))[0] if x in file_path_cols else x for x in pep_abundance.columns]
        pep_abundance = pep_abundance.rename(columns={'Modified.Sequence': 'Annotated Sequence'})

        #convert to str for IDs matrix
        pep_ID = pep_abundance.copy()
        cols = [col for col in pep_ID.columns if col != 'Accession']
        for col in cols:
            if pep_ID[col].dtype != 'object': # Check if not a string column
                pep_ID[col].replace(0, np.nan, inplace=True)
                # Replace all numerical values to ID
                pep_ID[col] = pep_ID[col].astype(str).str.replace("\d+\.\d+", "ID", regex=True)
     
    elif "PD" in process_app:
        peptide_table = pd.read_table(input2,low_memory=False)
        protein_table = pd.read_table(input1,low_memory=False)
        
        # filter Contaminant
        protein_table= protein_table[(protein_table[
            "Protein FDR Confidence: Combined"] == "High") &
                        ((protein_table["Master"] == "IsMasterProtein") | 
                         (protein_table["Master"] == "Master")) & 
                        (protein_table["Contaminant"] == False)]

        protein_table.rename(
            columns={'# Peptides': 'number of peptides'}, inplace=True)
        protein_table=protein_table.query(
            "`number of peptides` >= @min_unique_peptides")
        peptide_table= peptide_table[(peptide_table[
            'Contaminant'] == False) & (peptide_table["Confidence"]== "High")]

        meta_table = pd.read_table(input5,low_memory=False)
        #filter rows in meta table on File ID column if it is NaN
        meta_table = meta_table[meta_table['File ID'].notna()]

        # Replace single backslashes with forward slashes in the 'file_paths' column
        meta_table['File Name'] = meta_table['File Name'].str.replace('\\', '/', regex=False)
        # Apply a lambda function to extract file names without extensions
        meta_table['file_names'] = meta_table['File Name'].apply(lambda x: os.path.splitext(os.path.basename(x))[0])
        file_path_name_dict = dict(zip(meta_table['File ID'], meta_table['file_names']))
        run_name_list = pd.DataFrame({"Run Names": file_path_name_dict.values()})
        run_name_list['Run Identifier'] = run_name_list.index.to_series().apply(lambda x: prefix_letter + str(x))
        
        #format the read in table into three different tables: abundance, id and other_info
        prot_abundance = protein_table.filter(regex='Abundance:|Accession')
        prot_ID = protein_table.filter(regex='Found in Sample:|Accession')
        prot_other_info = protein_table.loc[:, ~protein_table.columns.str.contains('Found in Sample:|Abundance:')]
        

        pep_abundance = peptide_table.filter(regex='Abundance:|Annotated Sequence')
        pep_ID = peptide_table.filter(regex='Found in Sample:|Annotated Sequence')
        pep_other_info = peptide_table.loc[:, ~peptide_table.columns.str.contains('Found in Sample:|Abundance:')]

        prot_other_info["Source_File"] = input1
        pep_other_info["Source_File"] = input2

        #change column names to file/run names to our fileID

        new_dict = {"Abundance: " + key + ":": value for key, value in file_path_name_dict.items()}
        for item in [prot_abundance,pep_abundance]:
            # Generate a new column name mapping using the function
            column_name_mapping = generate_column_name_mapping(item.columns, new_dict)
            #TODO solving  A value is trying to be set on a copy of a slice from a DataFrame
            
            item.rename(columns = column_name_mapping, inplace = True)
            fileid_mapping = generate_column_name_mapping(item.columns, dict(zip(run_name_list["Run Names"],run_name_list["Run Identifier"])))
            item.rename(columns = fileid_mapping,inplace=True)
        

        new_dict = {"Found in Sample: " + key + ":": value for key, value in file_path_name_dict.items()}
        for item in [pep_ID,prot_ID]:
            # Generate a new column name mapping using the function
            column_name_mapping = generate_column_name_mapping(item.columns, new_dict)

            item.rename(columns = column_name_mapping, inplace = True)
            fileid_mapping = generate_column_name_mapping(item.columns, dict(zip(run_name_list["Run Names"],run_name_list["Run Identifier"])))

            item.rename(columns = fileid_mapping,inplace=True)

        # replace "High" to MS2 "Peak Found" to MBR, the rest to NaN for the ID tables
        replacements = {'High': 'MS2', 'Peak Found': 'MBR'}
        for column in run_name_list["Run Identifier"]:
            if column in pep_ID.columns:
                pep_ID[column] = pep_ID[column].apply(custom_replace)
    
            if column in prot_ID.columns:
                prot_ID[column] = prot_ID[column].apply(custom_replace)
    
    # get ID summary, tolly
    protein_ID_summary = sumIDs(prot_ID)
    peptide_ID_summary = sumIDs(pep_ID)


    
#     peptide_intensities = AbundanceMatrix(
#         peptide_table, maxLFQ_intensity=False, isProtein=False)
#     peptide_ID_matrix = toIDMatrix(peptide_intensities)
#     peptide_ID_summary = sumIDs(peptide_ID_matrix)

    #sets the processing app in run_name_list
    run_name_list["Processing App"] = process_app
    run_name_list["Analysis Name"] = analysis_file


    return {'run_metadata': run_name_list,
            'protein_other_info': prot_other_info,
            'peptide_other_info': pep_other_info,
            'protein_abundance': prot_abundance,
            'protein_ID_matrix': prot_ID,
            'protein_ID_Summary': protein_ID_summary,
            'peptide_abundance': pep_abundance,
            'peptide_ID_matrix': pep_ID,
            'peptide_ID_Summary': peptide_ID_summary,

            }  

# Define a custom replace function
def custom_replace(value):
    if value == 'High':
        return 'MS2'
    elif value == 'Peak Found':
        return 'MBR'
    else:
        return np.NaN

# Function to generate a new column name mapping based on the partial_column_name_mapping


In [112]:
def generate_column_name_mapping(columns, partial_column_name_mapping):
    column_name_mapping = {}
    for col in columns:
        for key, value in partial_column_name_mapping.items():
            if key in col:
                column_name_mapping[col] = value
                break
    return column_name_mapping

In [113]:
def sumIDs(IDMatrix):
    """_summarize the ID matrix infor into ID summary_
    

    Args:
        IDMatrix (_type_): _protein or pepetides matrix_
        0 Accession/Annotated Sequence 	run1 	run2 	run3 
        1 P023D12	MS2 	MBR 	NaN 
        2 P1222	NaN 	ID 	NaN 
    ID: means we don't know the ID mode

    Returns:
        _type_: _description_
                                      names  MS2_IDs  MBR_IDs  Total_IDs
0            10ng_QC_1_channel2 Intensity      NaN      NaN       3650
1            10ng_QC_2_channel1 Intensity      NaN      NaN       3604
....
    """
    # Select the columns of the runs
    columns = [col for col in IDMatrix.columns if not any(
        substring in col for substring in [
            'Accession', 'Annotated Sequence'])]
    #put each ID types into a list
    returnNames = []
    MS2_ID = []
    MBR_ID = []
    total_ID = []
    for eachColumn in columns:
        MS2_ID.append(len(IDMatrix[eachColumn][IDMatrix[eachColumn] == "MS2"]))
        MBR_ID.append(len(IDMatrix[eachColumn][IDMatrix[eachColumn] == "MBR"]))
        print(IDMatrix[eachColumn])
        total_ID_each = len(IDMatrix[eachColumn][IDMatrix[eachColumn] == "ID"])
        if total_ID_each ==0:
            total_ID_each = len(IDMatrix[eachColumn][
                IDMatrix[eachColumn] == "MS2"]) + len(IDMatrix[
                eachColumn][IDMatrix[eachColumn] == "MBR"])
        total_ID.append(total_ID_each)

    return pd.DataFrame({'names': columns,
                         'MS2_IDs': MS2_ID,
                         'MBR_IDs': MBR_ID,
                         'Total_IDs': total_ID})



In [114]:
def outer_join_data_objects(data_objects):
    '''
    Takes in a list of data objects as given by read_files and converts them to a single data object as given by read_files,
    protein info continues to show what was found on each original file, and so forth.
    '''

    first_file = True
    for eachDataObject in data_objects:
        print("***")
        if first_file:
            first_file = False
            final_data_object = eachDataObject
        else:
            final_data_object['run_metadata'] = pd.concat([final_data_object['run_metadata'],eachDataObject['run_metadata']]).reset_index()
            final_data_object['protein_other_info'] = pd.concat([final_data_object['protein_other_info'],eachDataObject['protein_other_info']]).reset_index()
            final_data_object['peptide_other_info'] = pd.concat([final_data_object['peptide_other_info'],eachDataObject['peptide_other_info']]).reset_index()
            final_data_object['protein_ID_Summary'] = pd.concat([final_data_object['protein_ID_Summary'],eachDataObject['protein_ID_Summary']]).reset_index()
            final_data_object['peptide_ID_Summary'] = pd.concat([final_data_object['peptide_ID_Summary'],eachDataObject['peptide_ID_Summary']]).reset_index()
            duplicates_found = False
            
            #loop through to see if there are any duplicate files
            for eachCol in final_data_object['protein_abundance'].loc[:, final_data_object['protein_abundance'].columns!='Accession'].columns:
                if eachCol in eachDataObject['protein_abundance'].columns:
                    duplicates_found = True
                else:
                    pass
            for eachCol in final_data_object['protein_ID_matrix'].loc[:, final_data_object['protein_ID_matrix'].columns!='Accession'].columns:
                if eachCol in eachDataObject['protein_ID_matrix'].columns:
                    duplicates_found = True
                else:
                    pass
            for eachCol in final_data_object['peptide_abundance'].loc[:, final_data_object['peptide_abundance'].columns!='Annotated Sequence'].columns:
                if eachCol in eachDataObject['peptide_abundance'].columns:
                    duplicates_found = True
                else:
                    pass
            for eachCol in final_data_object['peptide_ID_matrix'].loc[:, final_data_object['peptide_ID_matrix'].columns!='Annotated Sequence'].columns:
                if eachCol in eachDataObject['peptide_ID_matrix'].columns:
                    duplicates_found = True
                else:
                    pass     
            if duplicates_found:
                print("Error: files analyzed twice present!!!")
                quit()
                print("@#afio2q3")
            else:
                #merge keeping all proteins
                print("!!!!")
                final_data_object['protein_abundance'] = pd.merge(final_data_object['protein_abundance'],eachDataObject['protein_abundance'],how="outer")
                final_data_object['protein_ID_matrix'] = pd.merge(final_data_object['protein_ID_matrix'],eachDataObject['protein_ID_matrix'],how="outer")
                final_data_object['peptide_abundance'] = pd.merge(final_data_object['peptide_abundance'],eachDataObject['peptide_abundance'],how="outer")
                final_data_object['peptide_ID_matrix'] = pd.merge(final_data_object['peptide_ID_matrix'],eachDataObject['peptide_ID_matrix'],how="outer")
                
    return final_data_object

In [115]:
def filter_by_missing_values(data_object,
                             missing_value_thresh=33,
                             is_protein=True,
                             ignore_nan=False):
    """_Filter out proteins/peptides with missing values rate above the
    threshold_

    Args:
        data_object (_panada_): _dataframe contain data for one experimental
        condition_
        missing_value_thresh (int, optional): _description_. Defaults to 33.
        analysis_program (str, optional): _description_.
        ignore_nan: if filter intensity again with Nan threadshold, this 
        helps with the calcualting stdev step.

    Returns:
        _data_object_: _dictionary containing data for one experimental
         'abundances':        Accession  3_TrypsinLysConly_3A4_channel2
0     A0A096LP49                            0.00
1     A0A0B4J2D5                        89850.26
2         A0AVT1                        83055.87
    """
    if is_protein:
        name = "Accession"
        matrix_name = "protein_ID_matrix"
        other_info_name = "protein_other_info"
        abundance_name = "protein_abundance"
        
    else:
        name = "Annotated Sequence"
        matrix_name = "peptide_ID_matrix"
        other_info_name = "peptide_other_info"
        abundance_name = "peptide_abundance"

    protein_columns = data_object[matrix_name].assign(missingValues=0)

    i = 0
    # found all the proteins/peptides with missing values rate below
    # the threshold, pep_columns contains the remaining protein/peptide
    # in a pandas dataframe with $names as its column name
    for each_column in data_object[matrix_name].loc[
            :, ~data_object[matrix_name].columns.str.contains(
                name)].columns:
        # replace "nan" to np.nan
        protein_columns = protein_columns.replace({"nan": np.nan}) 
        protein_columns.loc[protein_columns[each_column].isnull(),
                             "missingValues"] += 1

        i += 1

    protein_columns = protein_columns.assign(missingValuesRate=(
        protein_columns["missingValues"] / i) * 100)
    
    protein_columns = protein_columns.query(
        "missingValuesRate < @missing_value_thresh")
    
    protein_columns = protein_columns.loc[:,
                                  protein_columns.columns.str.contains(name)]

    # filter the data_object with the remaining proteins/peptides names
    data_object[abundance_name] = protein_columns.merge(
        data_object[abundance_name])
    data_object[matrix_name] = protein_columns.merge(
        data_object[matrix_name])
    data_object[other_info_name] = protein_columns.merge(
        data_object[other_info_name])
    # In case there is mismatch between ID table and abundance table,
    # mannually remove the row with all NaN values
    # keep rows in data_object[abundance_name] where at least two values are 
    # not NaN(do this to all rows except the first row), otherwise can't
    # calculate the stdev
    if ignore_nan:
        data_object[abundance_name] = data_object[abundance_name].dropna(
            thresh=2, subset=data_object[abundance_name].columns[1:])
        # This will cause the veen diagram to be different from R program
    
    return data_object




In [116]:
def NormalizeToMedian(abundance_data, apply_log2=False):
    """_Normalizes each column by multiplying each value in that column with
    the median of all values in abundances (all experiments) and then dividing
    by the median of that column (experiment)._
    Args:
        abundance_data (_pd_): _description_
        apply_log2 (_bool_,): _apply log2 to all result_.
    Returns:
        _type_: _description_
        format:
         'abundances':        Accession  3_TrypsinLysConly_3A4_channel2
         A0A096LP49                    0.000000e+00
    """
    # all the columns/sample list
    columns = [col for col in abundance_data.select_dtypes(include=[
            np.number])]
    data_matrix = abundance_data[columns].values
    # replace 0 with nan
    data_matrix[data_matrix == 0] = np.nan
    medianOfAll = np.nanmedian(data_matrix)
    
    #normalize all median, all median/current run all protein median
    # apply log2 to all the values if apply_log2 is True
    if apply_log2:    
        for each_column in columns:
            abundance_data[each_column] = (
                np.log2(medianOfAll) * np.log2(abundance_data[each_column]) /
                np.log2(np.nanmedian(abundance_data[
                    each_column].replace(0, np.nan))))
    else:
        for each_column in columns:
            abundance_data[each_column] = (
                medianOfAll * abundance_data[each_column] /
                np.nanmedian(abundance_data[
                    each_column].replace(0, np.nan)))
    #TODO divide by zero error encountered in log2, temporarily set to 0
    abundance_data = abundance_data.replace([np.inf, -np.inf], 0)

    return abundance_data

In [117]:
def calculate_cvs(abundance_data):
    """_Calculate mean, stdev, cv for withn each protein/peptide abundance_

    Args:
        data_object (_type_): _full data frame_

    Returns:
        _type_: _df with Accession mean, stdev, cv for each protein/peptide_
    """
    if 'Accession' in abundance_data.columns:
        name = "Accession"
    if 'Annotated Sequence' in abundance_data.columns:
        name = "Annotated Sequence"
    abundance_data = abundance_data.assign(
        intensity=abundance_data.loc[:, ~abundance_data.columns.str.contains(
            name)].mean(axis=1, skipna=True),
        stdev=abundance_data.loc[:, ~abundance_data.columns.str.contains(
            name)].std(axis=1, skipna=True),
        CV=abundance_data.loc[:, ~abundance_data.columns.str.contains(name)].std(
            axis=1, skipna=True) / abundance_data.loc[
            :, ~abundance_data.columns.str.contains(name)].mean(
            axis=1, skipna=True) * 100)

    abundance_data = abundance_data.loc[:, [
            name, "intensity", "stdev", "CV"]]
    
    return abundance_data

In [118]:
def t_test_from_summary_stats(m1, m2, n1, n2, s1, s2, equal_var=False):
    """_Calculate T-test from summary using ttest_ind_from_stats from
    scipy.stats package_

    Args:
        m1 (_type_): _mean list of sample 1_
        m2 (_type_): mean list of sample 2_
        n1 (_type_): sample size list of sample 1_
        n2 (_type_): sample size list of sample 2_
        s1 (_type_): standard deviation list of sample 1_
        s2 (_type_): standard deviation list of sample 2_
        equal_var (_type_, optional): False would perform Welch's
        t-test, while set it to True would perform Student's t-test. Defaults
        to False.

    Returns:
        _type_: _list of P values_
    """

    p_values = []
    for i in range(len(m1)):
        _, p = ttest_ind_from_stats(
            m1[i], s1[i], n1[i], m2[i], s2[i], n2[i], equal_var=equal_var)
        p_values.append(p)

    return p_values

In [119]:
def CombineSharedInformation(infoObject1, infoObject2):
    """_Combine two infoObject into one_
    #TODO: there must be a better way to achieve this.
    """

    if infoObject1['meta']["quan_method"] == 'Protein':
        name = 'Accession'
    else:
        name = 'Annotated Sequence'

    infoObject = {'meta': None,
                  'run_name': None,
                  'protein_data': None,
                  'abundances': None,
                  'protein_ID_matrix': None,
                  'protein_ID_Summary': None}

    if infoObject1['meta'] == infoObject2['meta']:
        infoObject['meta'] = infoObject1['meta']
        infoObject['run_name'] = pd.concat(
            [infoObject1['run_name'], infoObject2['run_name']], axis=0)
        infoObject['protein_ID_Summary'] = pd.concat(
            [infoObject1['protein_ID_Summary'], infoObject2[
                'protein_ID_Summary']], axis=0)
        infoObject['protein_ID_matrix'] = pd.merge(
            infoObject1['protein_ID_matrix'], infoObject2[
                'protein_ID_matrix'], on=name)
        infoObject = pd.merge(
            infoObject1, infoObject2, on=name)
        infoObject['protein_data'] = pd.merge(infoObject1[
            'protein_data'].loc[:, infoObject1[
                'protein_data'].columns.str.contains(name)],
            infoObject2['protein_data'],
            on=name)
    else:
        infoObject = "ERROR: incompatible data types"

    return infoObject




In [120]:
def impute_knn(abundance_data, k=5):
    """_inpute missing value from neighbor values_

    Args:
        abundance_data (_type_): _description_
        k (int, optional): _number of neighbors used_. Defaults to 5.
    Returns:
        _type_: _description_
        TODO: this knn imputer produces slightly different results (about 4%)
        from the one in R. Need to figure out why
    """
    name = abundance_data.columns[0]

    names = abundance_data[name]
    # x = abundance_data.select_dtypes(include=['float64', 'int64'])
    # imputer = KNNImputer(n_neighbors=k)
    # x_imputed = pd.DataFrame(imputer.fit_transform(x), columns=x.columns)


    x = abundance_data.select_dtypes(include=['float', 'int'])
    imputer = KNNImputer(n_neighbors=k)
    x_imputed = imputer.fit_transform(x)
    x_imputed = pd.DataFrame(x_imputed, columns=x.columns)



    abundance_data.loc[:, x.columns] = x_imputed.values
    abundance_data[name] = names
    return abundance_data




In [121]:
def CalculatePCA(abundance_object, infotib,log2T = False):
    """_inpute PCA transformed and variance explained by each principal
    component_
    """
    name = abundance_object.columns[0]
    x = abundance_object

    sampleNames = x.columns[~x.columns.str.contains(
        name)].to_frame(index=False)

    if log2T: #apply log2 transformation
        x = np.log2(x.loc[:, ~x.columns.str.contains(name)].T.values)
    else:
        x = x.loc[:, ~x.columns.str.contains(name)].T.values
    # filter out columns with all zeros
    is_finite_col = np.isfinite(np.sum(x, axis=0))
    x_filtered = x[:, is_finite_col]

    
    # Instantiate PCA    
    pca = PCA()
    #
    # Determine transformed features
    #
    x_pca = pca.fit_transform(x_filtered)
    #
    # Determine explained variance using explained_variance_ration_ attribute
    #
    exp_var_pca = pca.explained_variance_ratio_
    #
    # Cumulative sum of eigenvalues; This will be used to create step plot
    # for visualizing the variance explained by each principal component.
    #
    cum_sum_eigenvalues = np.cumsum(exp_var_pca)
    #
    # convert numpy array to pandas dataframe for plotting
    
    pca_panda = pd.DataFrame(x_pca, columns=[
        'PC' + str(i+1) for i in range(x_pca.shape[1])])
    # add sample names to the dataframe
    pca_panda = pd.concat(
        [infotib, pca_panda], axis=1, join='inner')
    
    return pca_panda, exp_var_pca




In [122]:
def filter_by_name(data_dict, runname_list):
    """_Filter the data_dict based on runname_list, only keep the columns
    of the data_dict that are in the runname_list_
    Args:

    Returns:
        _type_: _description_
    """
    filtered_data = {}
   # filtered_data["meta"] = data_dict["meta"]
    runname_list.extend(["Annotated Sequence","Accession"])
    
    # filter in good ones
    filtered_data["run_metadata"] = [item for item in data_dict[
        "run_metadata"] if item in runname_list]    
    filtered_data["protein_abundance"] = data_dict["protein_abundance"][[
        col for col in data_dict["protein_abundance"].columns if any(
            word in col for word in runname_list)]]
    filtered_data["peptide_abundance"] = data_dict["peptide_abundance"][[
        col for col in data_dict["peptide_abundance"].columns if any(
            word in col for word in runname_list)]]
    filtered_data["protein_other_info"] = data_dict["protein_other_info"][[
        col for col in data_dict["protein_other_info"].columns if any(
            word in col for word in runname_list)]]
    filtered_data["peptide_other_info"] = data_dict["peptide_other_info"][[
        col for col in data_dict["peptide_other_info"].columns if any(
            word in col for word in runname_list)]]
    filtered_data["protein_ID_matrix"] = data_dict["protein_ID_matrix"][[
        col for col in data_dict["protein_ID_matrix"].columns if any(
            word in col for word in runname_list)]]
    filtered_data["protein_ID_Summary"] = data_dict["protein_ID_Summary"][
        data_dict["protein_ID_Summary"]["names"].isin(
            runname_list)]
    filtered_data["peptide_ID_Summary"] = data_dict["peptide_ID_Summary"][
        data_dict["peptide_ID_Summary"]["names"].isin(
            runname_list)]
    return filtered_data

In [123]:
def ID_plots(data_object, plot_options, saved_settings, username=None):
    """_Prepare data for creating protein peptide identification bar
    plot_

    Args:
        data_dict (_type_): _description_
    """
    # Create an empty dictionary to store the group names and filters
    group_names = []
    for item in GROUP_NAME:
        if saved_settings[item]:
            group_names.append(saved_settings[item])

    # import the data
    group_dict = {}

    # filter runs into different groups
    i = 1
    runname_list = []  # contain list of run names list for each groups
    for eachGroup in group_names:
        runname_sublist = [d.get('option')
                           for d in saved_settings[f"group{i}_record"]]

        group_dict[eachGroup] = filter_by_name(
            data_object,
            list(runname_sublist))  # prevent the list from being changed
        runname_list.append(runname_sublist)
        i += 1

    # create ID plots
    # allIDs table will be used to store all experiment name, ID types (
    # protein, peptide, MS2 and MS1 based), conditions and IDs numbers
    allIDs = pd.DataFrame(
        columns=["Names", "ID_Type", "ID_Mode", "Conditions", "IDs"])

    # loop through each group and extract IDs, put them into allIDs table
    for eachCondition in group_names:
        # Protein ID summary
        for index, row in group_dict[eachCondition][
                "protein_ID_Summary"].iterrows():
            for item in ["MS2_IDs",
                         "MBR_IDs",
                         "Total_IDs"]:
                if not pd.isna(group_dict[eachCondition][
                        "protein_ID_Summary"].at[index, item]):
                    # if the row with the item column is not empty,
                    # add it to allIDs table.
                    allIDs = pd.concat(
                        [allIDs,
                         pd.DataFrame(
                             [[group_dict[eachCondition][
                                 "protein_ID_Summary"].at[index, "names"],
                              "protein",
                               item,
                               eachCondition,
                               group_dict[eachCondition][
                                 "protein_ID_Summary"].at[index, item]]],
                             columns=["Names",
                                      "ID_Type",
                                      "ID_Mode",
                                      "Conditions",
                                      "IDs"])],
                        ignore_index=True)
        # Peptide ID summary
        for index, row in group_dict[eachCondition][
                "peptide_ID_Summary"].iterrows():
            for item in ["MS2_IDs",
                         "MBR_IDs",
                         "Total_IDs"]:
                if not pd.isna(group_dict[eachCondition][
                        "peptide_ID_Summary"].at[index, item]):
                    allIDs = pd.concat(
                        [allIDs,
                         pd.DataFrame(
                             [[group_dict[eachCondition][
                                 "peptide_ID_Summary"].at[index, "names"],
                              "peptide",
                               item,
                               eachCondition,
                               group_dict[eachCondition][
                                 "peptide_ID_Summary"].at[index, item]]],
                             columns=["Names",
                                      "ID_Type",
                                      "ID_Mode",
                                      "Conditions",
                                      "IDs"])],
                        ignore_index=True)
    # ######################allIDs format###################
    # name	ID_Type	ID_Mode	Conditions	IDs
    # file1	peptide	MS2_IDs	experimetn 1	xxxxx
    # file2	protein	MBR_IDs	experiment 2	xxxx
    # file3	peptide	Total_IDs	experiment 3	xxx
    #######################################################
    # Calcuate mean, standard deviation and number of replicates for each

    # choose protein or peptide
    if plot_options["plot_type"] == "1":  # Protein ID
        allIDs = allIDs[allIDs["ID_Type"] == "protein"]
    elif plot_options["plot_type"] == "2":  # Peptide ID
        allIDs = allIDs[allIDs["ID_Type"] == "peptide"]

    # choose total, MS2 or stacked
    if plot_options["ID mode"] == "MS2":  # MS2 ID
        allIDs = allIDs[allIDs["ID_Mode"] == "MS2_IDs"]
    elif plot_options["ID mode"] == "stacked":  # total separated
        pass
    else:
        # total ID combined, if not already summed (key exist), sum them
#         if allIDs[allIDs["ID_Mode"] == "Total_IDs"].empty:
#             grouped = allIDs.groupby('name').agg(
#                 {'IDs': 'sum', 'ID_Type': 'first', 'Conditions': 'first'})
#             grouped = grouped.reset_index()
#             grouped["ID_Mode"] = "Total_IDs"
#             allIDs = grouped
        allIDs = allIDs[allIDs["ID_Mode"] == "Total_IDs"]

    toPlotIDs = allIDs.groupby(["ID_Mode", "Conditions"]).agg({
        'IDs': ['mean', 'std', 'count'], 'ID_Type': 'first', })

    # rename the columns
    toPlotIDs.columns = ['IDs', 'stdev', 'n', 'ID_Type']
    # reset the index after grouping
    toPlotIDs = toPlotIDs.reset_index()
    # calculate the confidence interval based on 95%confidence interval`
    toPlotIDs["confInt"] = t.ppf(0.975, toPlotIDs['n']-1) * \
        toPlotIDs['stdev']/np.sqrt(toPlotIDs['n'])

    return plot_IDChart_plotly(toPlotIDs,
                               username=username,
                               plot_options=plot_options)
def plot_IDChart_plotly(ID_data,
                        username=None,
                        plot_options=None):
    """_Plot the ID bar plot for the given data_

    Args:
        ID_data (_type_): _description_
        username (str, optional): _description_. Defaults to "test".
        plot_options (_type_, optional): _description_. Defaults to None.

    Returns:
        _type_: _description_
    """

    plot_div = None
    CSV_link = None
    SVG_link = None

        
    if plot_options["ID mode"] != "stacked":

        # plot options
        # error bar
        if plot_options["error bar"] == "stdev":
            error_bars = "stdev"
        elif plot_options["error bar"] == "ci95":
            error_bars = "confInt"
        else:
            error_bars = None

        # mean label
        if plot_options["mean label"] == "True" or \
                plot_options["mean label"] == True:
            total_labels = [{"x": x, "y": total*1.15, "text": str(
                int(total)), "showarrow": False} for x, total in zip(
                    ID_data["Conditions"], ID_data["IDs"])]
        else:
            total_labels = []   # no mean labels



        # create the plot
        fig = px.bar(ID_data,
                     x="Conditions",
                     y="IDs",
                     error_y=error_bars,
                     color="Conditions",
                     color_discrete_sequence=plot_options["color"],
                     width=plot_options["width"],
                     height=plot_options["height"],
                     )
        fig.update_layout(xaxis_title=plot_options["X Title"],
                          yaxis_title=plot_options["Y Title"],
                          annotations=total_labels,
                          font=plot_options["font"]
                          )
    if WRITE_OUTPUT:    
        # export the data to csv for user downloading
        data_dir = os.path.join(APPFOLDER, "csv/")
        # create the directory if it does not exist
        if not os.path.exists(data_dir):
            Path(data_dir).mkdir(parents=True)

        # export the data to csv
        ID_data.to_csv(os.path.join(
            data_dir, f"{username}_ID_data.csv"), index=False)
        # create the link for downloading the data
        CSV_link = f"/files/{url_base}/csv/" \
            f"{username}_ID_data.csv"

        # add SVG download link

        SVG_link = f"/files/{url_base}/images/" \
            f"{username}_ID_Bar_Plot.svg"

        img_dir = os.path.join(APPFOLDER, "images/")
        if not os.path.exists(img_dir):
            Path(img_dir).mkdir(parents=True)

        fig.write_image(os.path.join(
            img_dir, f"{username}_ID_Bar_Plot.svg"))


    else:
        pass
    return fig, CSV_link, SVG_link

In [124]:
# CV Violin plots ###
def CV_plots(data_object, plot_options, saved_settings, username=None):
    """_Prepare data for creating protein CV violin plots_
    """
    group_names = []
    for item in GROUP_NAME:
        if saved_settings[item]:
            group_names.append(saved_settings[item])

    # import the data
    group_dict = {}

    # filter runs into different groups
    i = 1
    runname_list = []  # contain list of run names list for each groups
    for eachGroup in group_names:
        runname_sublist = [d.get('option')
                           for d in saved_settings[f"group{i}_record"]]

        group_dict[eachGroup] = filter_by_name(
            data_object,
            list(runname_sublist))  # prevent the list from being changed
        runname_list.append(runname_sublist)
        i += 1

    # create a dictionary to store the intensity data
    Intensity_dict = {}

    for eachGroup in group_names:
        current_condition_data = filter_by_missing_values(
            group_dict[eachGroup])
        Intensity_dict[eachGroup] = NormalizeToMedian(
            current_condition_data["protein_abundance"])

    all_cvs = pd.DataFrame()

    for eachGroup in Intensity_dict:
        current = calculate_cvs(
            Intensity_dict[eachGroup]).assign(Conditions=eachGroup)
        all_cvs = pd.concat([all_cvs, current], ignore_index=True)

    # ######################all_CVs format###################
#      Accession     intensity          stdev          CV   Conditions
# 0       A6NHR9  3.248547e+06  672989.819300   20.716643    DDMandDTT
# 1       A8MTJ3  5.031539e+05  195535.383583   38.861944    DDMandDTT
# 2       E9PAV3  5.330290e+05  161385.491163   30.277056    DDMandDT
    #######################################################

    return plot_CV_violin(allCVs=all_cvs,
                          username=username,
                          plot_options=plot_options)


def plot_CV_violin(allCVs,
                   username=None,
                   plot_options=None,
                   ):
    """_Plot the CV violin plot for the given data._

    Args:
        allCVs (_type_): _description_
        username (_type_, optional): _description_. Defaults to None.
        plot_options (_type_, optional): _description_. Defaults to None.

    Returns:
        _type_: _description_
    """
    plot_div = None
    CSV_link = None
    SVG_link = None

    allCVs_summary = allCVs.groupby(["Conditions"]).agg(
        {'CV': ['median', 'mean']}).reset_index()
    allCVs_summary.columns = ["Conditions", 'meds', 'CoVar']
    # mean label
    if plot_options["mean label"] == "True" or \
            plot_options["mean label"] == True:
        total_labels = [{"x": x, "y": total*1.15, "text": str(
            round(total,1)), "showarrow": False} for x, total in zip(
            allCVs_summary["Conditions"], allCVs_summary["meds"])]
    else:
        total_labels = []   # no mean labels

    
    # create the interactive plot
    fig = px.violin(allCVs,
                    x="Conditions",
                    y='CV',
                    color="Conditions",
                    box=bool(plot_options["box"]),
                    violinmode=plot_options["violinmode"], hover_data=[
                        "Conditions", 'CV'],
                    color_discrete_sequence=plot_options["color"],
                    width=plot_options["width"],
                    height=plot_options["height"],
                    )

    fig.update_layout(
        yaxis=dict(title=plot_options["Y Title"],
                   range=plot_options["ylimits"]),
        font=plot_options["font"],
        xaxis=dict(title=plot_options["X Title"]),
        showlegend=True,
        annotations=total_labels
    )

    if WRITE_OUTPUT:        
        # create the file for donwnload
        img_dir = os.path.join(APPFOLDER, "images/")
        if not os.path.exists(img_dir):
            Path(img_dir).mkdir(parents=True)

        fig.write_image(os.path.join(
            img_dir, f"{username}_CV_Violin_Plot.svg"))
        
        # create the download CSV and its link
        data_dir = os.path.join(APPFOLDER, "csv/")
        if not os.path.exists(data_dir):
            Path(data_dir).mkdir(parents=True)
        allCVs.to_csv(os.path.join(
            data_dir, f"{username}_all_CV.csv"), index=False)
        CSV_link = f"/files/{url_base}/csv/" \
            f"{username}_all_CV.csv"

        # download SVG link
        SVG_link = f"/files/{url_base}/images/" \
            f"{username}_CV_Violin_Plot.svg"


    return fig, CSV_link, SVG_link




In [125]:
def venns_plots(data_object, plot_options, saved_settings, username=None):
    """_Prepare data for creating ID veens plots (up to three groups)_
    """
    group_names = []

    # no compare groups is provided, compare first two
    if not plot_options["compare groups"] or \
            plot_options["compare groups"] == "[]" or \
            not isinstance(plot_options["compare groups"], list):
        for item in GROUP_NAME[:2]:
            if saved_settings[item]:
                group_names.append(saved_settings[item])
    else:  # compare groups (list of 2/3 numbers, start with 0) is provided,
        for n in range(len(GROUP_NAME)):
            if n in plot_options["compare groups"] and saved_settings[
                    GROUP_NAME[n]]:
                group_names.append(saved_settings[GROUP_NAME[n]])
    # import the data
    group_dict = {}

    # filter runs into different groups
    i = 0
    runname_list = []  # contain list of run names list for each groups
    for eachGroup in group_names:
        runname_sublist = [d.get('option')
                           for d in saved_settings[
            f"group{plot_options['compare groups'][i]+1}_record"]]
        group_dict[eachGroup] = filter_by_name(
            data_object,
            list(runname_sublist))  # prevent the list from being changed
        runname_list.append(runname_sublist)
        i += 1
    data_set = []
    labels_set = []
    
    for eachGroup in group_names:
        
        current_condition_data = filter_by_missing_values(
            group_dict[eachGroup])

        data_set.append(
            set(current_condition_data['protein_abundance']['Accession'].unique()))
        labels_set.append(eachGroup)

    return venn_to_plotly(
        data_set,
        labels_set,
        plot_options=plot_options,
        username=username)


def venn_to_plotly(L_sets,
                   L_labels=None,
                   plot_options=None,
                   username=None):
    """_Creates a venn diagramm from a list of
    sets and returns a plotly figure_
    """
    CSV_link = None
    SVG_link = None

    # get number of sets
    n_sets = len(L_sets)

    # choose and create matplotlib venn diagramm
    if n_sets == 2:
        if L_labels and len(L_labels) == n_sets:
            v = venn2(L_sets, L_labels)
        else:
            v = venn2(L_sets)
    elif n_sets == 3:
        if L_labels and len(L_labels) == n_sets:
            v = venn3(L_sets, L_labels)
        else:
            v = venn3(L_sets)
    # supress output of venn diagramm
    # plt.show()
    plt.close()

    # Create empty lists to hold shapes and annotations
    L_shapes = []
    L_annotation = []

    # Define color list for sets
    L_color = plot_options["color"]

    # Create empty list to make hold of min and max values of set shapes
    L_x_max = []
    L_y_max = []
    L_x_min = []
    L_y_min = []

    for i in range(0, n_sets):

        # create circle shape for current set

        shape = go.layout.Shape(
            type="circle",
            xref="x",
            yref="y",
            x0=v.centers[i][0] - v.radii[i],
            y0=v.centers[i][1] - v.radii[i],
            x1=v.centers[i][0] + v.radii[i],
            y1=v.centers[i][1] + v.radii[i],
            fillcolor=L_color[i],
            line_color=L_color[i],
            opacity=plot_options["opacity"]
        )

        L_shapes.append(shape)

        # create set label for current set
        try:
            anno_set_label = go.layout.Annotation(
                xref="x",
                yref="y",
                x=v.set_labels[i].get_position()[0],
                y=v.set_labels[i].get_position()[1],
                text=v.set_labels[i].get_text(),
                showarrow=False
            )

            L_annotation.append(anno_set_label)

            # get min and max values of current set shape
            L_x_max.append(v.centers[i][0] + v.radii[i])
            L_x_min.append(v.centers[i][0] - v.radii[i])
            L_y_max.append(v.centers[i][1] + v.radii[i])
            L_y_min.append(v.centers[i][1] - v.radii[i])
        except Exception as err:
            print(f"No set labels found {err}")

    # determine number of subsets
    n_subsets = sum([scipy.special.binom(n_sets, i+1)
                     for i in range(0, n_sets)])

    for i in range(0, int(n_subsets)):
        try:

            # create subset label (number of common elements for current subset

            anno_subset_label = go.layout.Annotation(
                xref="x",
                yref="y",
                x=v.subset_labels[i].get_position()[0],
                y=v.subset_labels[i].get_position()[1],
                text=v.subset_labels[i].get_text(),
                showarrow=False
            )

            L_annotation.append(anno_subset_label)
        except Exception as err:
            print(f"No set labels found {err}")
    # define off_set for the figure range
    off_set = 0.2

    # get min and max for x and y dimension to set the figure range
    x_max = max(L_x_max) + off_set
    x_min = min(L_x_min) - off_set
    y_max = max(L_y_max) + off_set
    y_min = min(L_y_min) - off_set

    # create plotly figure

    fig = go.Figure()

    # set xaxes range and hide ticks and ticklabels
    fig.update_xaxes(
        range=[x_min, x_max],
        showticklabels=False,
        ticklen=0
    )

    # set yaxes range and hide ticks and ticklabels
    fig.update_yaxes(
        range=[y_min, y_max],
        scaleanchor="x",
        scaleratio=1,
        showticklabels=False,
        ticklen=0
    )

    # set figure properties and add shapes and annotations
    fig.update_layout(
        plot_bgcolor='white',
        margin=dict(b=0, l=10, pad=0, r=10, t=40),
        width=800,
        height=400,
        shapes=L_shapes,
        annotations=L_annotation,
        title=dict(text=plot_options["title"], x=0.5, xanchor='center')
    )
    if WRITE_OUTPUT:
        # SVG file link
        SVG_link = f"/files/{url_base}/images/" \
            f"{username}_ID_venns_Plot.svg"

        # create the file for donwnload
        img_dir = os.path.join(APPFOLDER, "images/")
        if not os.path.exists(img_dir):
            Path(img_dir).mkdir(parents=True)

        fig.write_image(os.path.join(
            img_dir, f"{username}_ID_venns_Plot.svg"))

    return fig, CSV_link, SVG_link


In [126]:
# ###Volcano plots####
def volcano_plots(data_object,  plot_options, saved_settings, username=None):
    """_Prepare data for creating intensity volcano plots (two groups)_
    """
    group_names = []

    # no compare groups is provided, compare first two
    if not plot_options["compare groups"] or \
            plot_options["compare groups"] == "[]" or \
            not isinstance(plot_options["compare groups"], list):
        for item in GROUP_NAME[:2]:
            if saved_settings[item]:
                group_names.append(saved_settings[item])
    else:  # compare groups (list of two, start with 0) is provided,
        for n in range(len(GROUP_NAME)):
            if n in plot_options["compare groups"] and saved_settings[
                    GROUP_NAME[n]]:
                group_names.append(saved_settings[GROUP_NAME[n]])
    # import the data
    group_dict = {}

    # filter runs into different groups
    i = 0
    runname_list = []  # contain list of run names list for each groups
    for eachGroup in group_names:
        runname_sublist = [d.get('option')
                           for d in saved_settings[
            f"group{plot_options['compare groups'][i]+1}_record"]]
        group_dict[eachGroup] = filter_by_name(
            data_object,
            list(runname_sublist))  # prevent the list from being changed
        runname_list.append(runname_sublist)
        i += 1
    # create a dictionary to store the intensity data
    Intensity_dict = {}

    for eachGroup in group_names:
        current_condition_data = filter_by_missing_values(
            group_dict[eachGroup])
        Intensity_dict[eachGroup] = NormalizeToMedian(
            current_condition_data["protein_abundance"],apply_log2=True)
    group1 = group_names[0]
    group2 = group_names[1]
    # calculate mean, standard deviation, and the number of non-null
    # elements for each row/protein
    group1Data = (Intensity_dict[group1]
                  .assign(group1_Intensity=Intensity_dict[group1].drop(
        columns=['Accession']).mean(axis=1),
        group1_stdev=Intensity_dict[group1].drop(
                      columns=['Accession']).std(axis=1),
        group1_num=Intensity_dict[group1].drop(
                      columns=['Accession']).shape[1] - Intensity_dict[
                      group1].isna().sum(axis=1))
                  .loc[:, ['group1_Intensity',
                           'group1_stdev',
                           'group1_num',
                           'Accession']])
    """ group1Data
            group1_Intensity  group1_stdev  group1_num   Accession
    0        2.824766e+05  1.708060e+05          15  A0A0B4J2D5
    1        2.650998e+06  6.259645e+05          15      A2RUR9
    2        1.973150e+05  5.645698e+04          15      A8MTJ3
    3        2.524020e+05  1.355699e+05          15      A8MWD9
    """
    group1Prots = group1Data.loc[:, ['Accession']]

    group2Data = (Intensity_dict[group2]
                  .assign(group2_Intensity=Intensity_dict[group2].drop(
        columns=['Accession']).mean(axis=1),
        group2_stdev=Intensity_dict[group2].drop(
                      columns=['Accession']).std(axis=1),
        group2_num=Intensity_dict[group2].drop(
                      columns=['Accession']).shape[1]
        - Intensity_dict[group2].isna().sum(axis=1))
        .loc[:, ['group2_Intensity',
                 'group2_stdev',
                 'group2_num',
                 'Accession']])
    # find common proteins
    commonProts = (group2Data.loc[:, ['Accession']]
                   .merge(group1Prots, on='Accession', how='inner'))
    # only leave common proteins
    group2Data = (group2Data
                  .merge(commonProts, on='Accession', how='inner'))
    group1Data = (group1Data
                  .merge(commonProts, on='Accession', how='inner'))

    group2Median = group2Data['group2_Intensity'].median(
        numeric_only=True)
    group1Median = group1Data['group1_Intensity'].median(
        numeric_only=True)
    allmedian = (group2Data['group2_Intensity']
                 .append(group1Data['group1_Intensity'])).median(
        numeric_only=True)

    # calculate the ratio between two group median,
    # will be used to normalize them
    if (Intensity_dict[group1].shape[1] > 3 and
        Intensity_dict[group2].shape[1] > 3 and
            group2 != group1):
        ratio1 = allmedian / group1Median
        ratio2 = allmedian / group2Median

        # merge these two set of data together, adjust groups with ratio to 
        # median of all. Calculate pOriginal, p, significant
        # pOriginal is a numpy array or list of p-values
        # method is the method to be used for adjusting the p-values
        volcanoData = (group2Data
                       .merge(group1Data, on='Accession', how='inner'))

        volcanoData = (volcanoData
                       .assign(group1_Intensity=lambda x: volcanoData[
                           'group1_Intensity'] * ratio1))
        volcanoData = (volcanoData
                       .assign(group2_Intensity=lambda x: volcanoData[
                           'group2_Intensity'] * ratio2))

        volcanoData = (volcanoData
                       .assign(
                           pOriginal=t_test_from_summary_stats(
                               m1=volcanoData['group2_Intensity'],
                               m2=volcanoData['group1_Intensity'],
                               s1=volcanoData['group2_stdev'],
                               s2=volcanoData['group1_stdev'],
                               n1=volcanoData['group2_num'],
                               n2=volcanoData['group1_num'])))
        # filter out rows in volcanoData that have pOriginal == nan
        # if pOriginal is nan, then the p value will be nan
        volcanoData = volcanoData[volcanoData['pOriginal'].notna()]
        volcanoData = (volcanoData
                       .assign(p=multipletests(volcanoData[
                           "pOriginal"], method='fdr_bh')[1]))

        volcanoData = (volcanoData
                       .assign(significant=(abs(volcanoData[
                           'group2_Intensity'] - volcanoData[
                           'group1_Intensity']) > 1)
                           & (volcanoData['p'] < 0.05)))

        # add upRegulated, downRegulated, and notRegulated columns
        volcanoData = volcanoData.assign(upRegulated=lambda x: (
            volcanoData["group2_Intensity"] - volcanoData[
                "group1_Intensity"] > 1) & (volcanoData['significant']))

        volcanoData = volcanoData.assign(downRegulated=lambda x: (
            volcanoData["group2_Intensity"]-volcanoData[
                "group1_Intensity"] < -1) & (volcanoData['significant']))
        volcanoData = volcanoData.assign(notRegulated=lambda x: (abs(
           volcanoData["group2_Intensity"]-volcanoData[
                "group1_Intensity"]) <= 1) & (~volcanoData['significant']))
        return plot_volcano_colored(
            volcanoData,
            label=f"({group2}/{group1})",
            plot_options=plot_options,
            username=username,
        )


def plot_volcano_colored(allData,
                         label,
                         plot_options=None,
                         username=None,):
    CSV_link = None
    SVG_link = None
    total_labels = []
    left = "group1_Intensity"
    right = "group2_Intensity"
    downData = allData[allData['downRegulated']
                       == True]
    upData = allData[allData['upRegulated'] == True]

    fig = px.scatter(
        width=plot_options["width"],
        height=plot_options["height"],)
    if allData.shape[0] != 0:
        fig.add_scatter(x=allData[right]-allData[left],
                        y=-np.log10(allData["p"]),
                        text=allData["Accession"],
                        mode="markers", marker=dict(
                            color=plot_options["all color"]))
    if downData.shape[0] != 0:
        fig.add_scatter(x=downData[right]-downData[left],
                        y=-np.log10(downData["p"]),
                        text=downData["Accession"],
                        mode="markers",
                        marker=dict(color=plot_options["down color"]))
    if upData.shape[0] != 0:
        fig.add_scatter(x=upData[right]-upData[left],
                        y=-np.log10(upData["p"]),
                        text=upData["Accession"],
                        mode="markers",
                        marker=dict(color=plot_options["up color"]))
        fig.update_traces(
            mode="markers",
            hovertemplate="%{text}<br>x=: %{x}"
            " <br>y=: %{y}")
    fig.add_hline(y=-np.log10(0.05))
    fig.add_vline(x=-1)
    fig.add_vline(x=1)
    if plot_options["title"] != "" or plot_options["title"] is not None:
        plot_title = plot_options["title"] + " " + label
    else:
        plot_title = None
    if not plot_options["xlimits"] or plot_options["xlimits"] == "[]" or \
            not isinstance(plot_options["xlimits"], list):
        xlimits = None
    else:
        xlimits = plot_options["xlimits"]

    if not plot_options["ylimits"] or plot_options["ylimits"] == "[]" or \
            not isinstance(plot_options["ylimits"], list):
        ylimits = None
    else:
        ylimits = plot_options["ylimits"]

    fig.update_layout(
        font=plot_options["font"],

        showlegend=False,
        title=plot_title,
        xaxis=dict(title=dict(
            text=plot_options["X Title"]), range=xlimits),
        yaxis=dict(title=dict(
            text=plot_options["Y Title"]), range=ylimits),
        annotations=total_labels

    )

    if WRITE_OUTPUT:
        # create the file for donwnload
        img_dir = os.path.join(APPFOLDER, "images/")
        if not os.path.exists(img_dir):
            Path(img_dir).mkdir(parents=True)

        fig.write_image(os.path.join(
            img_dir, f"{username}_abundance_volcano_Plot.svg"))
        # create the download CSV and its link

        data_dir = os.path.join(APPFOLDER, "csv/")
        if not os.path.exists(data_dir):
            Path(data_dir).mkdir(parents=True)
        allData.to_csv(os.path.join(
            data_dir, f"{username}_up_down_regulated_volcano.csv"),
            index=False)
        CSV_link = f"/files/{url_base}/csv/" \
            f"{username}_up_down_regulated_volcano.csv"

        # download SVG link
        SVG_link = f"/files/{url_base}/images/" \
            f"{username}_abundance_volcano_Plot.svg"
    return fig, CSV_link, SVG_link


In [127]:
# ###PCA plots####
def PCA_plots(data_object, plot_options, saved_settings,username=None):
    """_Prepare data for creating intensity PCA plots (two groups)_
    """
    group_names = []

    # no compare groups is provided, compare first two
    if not plot_options["compare groups"] or \
            plot_options["compare groups"] == "[]" or \
            not isinstance(plot_options["compare groups"], list):
        for item in GROUP_NAME[:2]:
            if saved_settings[item]:
                group_names.append(saved_settings[item])
    else:  # compare groups (list of two, start with 0) is provided,
        for n in range(len(GROUP_NAME)):
            if n in plot_options["compare groups"] and saved_settings[
                    GROUP_NAME[n]]:
                group_names.append(saved_settings[GROUP_NAME[n]])
    # import the data
    group_dict = {}

    # filter runs into different groups
    i = 0
    runname_list = []  # contain list of run names list for each groups
    for eachGroup in group_names:
        runname_sublist = [d.get('option')
                           for d in saved_settings[f"group{plot_options['compare groups'][i]+1}_record"]]
        group_dict[eachGroup] = filter_by_name(
            data_object,
            list(runname_sublist))  # prevent the list from being changed
        runname_list.append(runname_sublist)
        i += 1
    all_runs =[item for sublist in runname_list for item in sublist]

    # combined the data after filtering
    #  missing values and log2 transformation/normalization
    combined_infodata = pd.DataFrame() # store run names and group names
    combined_pcaData = pd.DataFrame() # store normalized data and protein names
    for eachGroup in group_names:

        current_condition_data = filter_by_missing_values(
            group_dict[eachGroup])
        normalized_data = NormalizeToMedian(
             current_condition_data["protein_abundance"],apply_log2=True)
        print(current_condition_data)
        toFileDict = dict(zip(data_object["run_metadata"]["Run Identifier"],data_object["run_metadata"]["Run Names"]))
        toFileDict = generate_column_name_mapping(normalized_data.columns, toFileDict)
        combined_infodata= pd.concat([combined_infodata, pd.DataFrame({
            "Sample_Groups": normalized_data
            .drop(
                "Accession", axis=1).rename(columns = toFileDict).columns,
            "Type": eachGroup})])
        if combined_pcaData.empty:
            combined_pcaData = normalized_data
        else:
            combined_pcaData = pd.merge(combined_pcaData, normalized_data)

    #normalize the data
    # using ratio of current group median value divide by the all groups median 
    # to create a scaling factor magicNUm to scale the each group
    for n in range(len(group_names)): 
        magicNum =np.nanmedian(combined_pcaData[runname_list[
            n]].dropna(how='all').to_numpy()) /\
                np.nanmedian(combined_pcaData[
            all_runs].dropna(how='all').to_numpy()) 
        for col in combined_pcaData[runname_list[
                n]].columns:
            combined_pcaData[col] = combined_pcaData[col]/magicNum


    #performs k-Nearest Neighbors imputation to fill in any missing values
    combined_pcaData = impute_knn(combined_pcaData)
    combined_infodata.reset_index(drop=True, inplace=True)
    

    # perform PCA transform
    combined_pcaData, exp_var_pca = CalculatePCA(combined_pcaData,
                                                     combined_infodata)

    return plot_PCA_plotly(combined_pcaData,
                           exp_var_pca,
                           plot_options=plot_options,
                           username=username,
                           )


def plot_PCA_plotly(pca_panda,
                    exp_var_pca,
                    plot_options=None,
                    username=None,):

    CSV_link = None
    SVG_link = None

    # Assuming pca_data is a pandas dataframe containing PCA results
    # and "Type" is a column in the dataframe indicating the type of sample
    if not plot_options["xlimits"] or plot_options["xlimits"] == "[]" or \
            not isinstance(plot_options["xlimits"], list):
        xlimits = None
    else:
        xlimits = plot_options["xlimits"]

    if not plot_options["ylimits"] or plot_options["ylimits"] == "[]" or \
            not isinstance(plot_options["ylimits"], list):
        ylimits = None
    else:
        ylimits = plot_options["ylimits"]

    fig = px.scatter(pca_panda,
                     x='PC1',
                     y='PC2',
                     color="Type",
                     text="Sample_Groups",
                     symbol="Type",
                     color_discrete_sequence=plot_options["color"],

                     symbol_sequence=plot_options["symbol"],
                     size_max=30,
                     labels={'PC1': f'PC1 ({round(exp_var_pca[0]*100,2)}%)',
                             'PC2': f'PC2 ({round(exp_var_pca[1]*100,2)}%)',
                             'Type': 'Sample Type'}, title='PCA Plot',
                     width=plot_options["width"],
                     height=plot_options["height"],)

    fig.update_traces(
        mode="markers",
        marker=dict(size=plot_options["marker_size"],),
        hovertemplate="%{text}<br>PC1: %{x} <br>PC2: %{y}")
    fig.update_layout(
        # plot_bgcolor="rgba(0, 0, 0, 0)",
        # paper_bgcolor="rgba(0, 0, 0, 0)",
        font=plot_options["font"],
        title=plot_options["title"],
        xaxis=dict(linecolor='black',
                   showticklabels=False, mirror=True, range=xlimits),
        yaxis=dict(linecolor='black',
                   showticklabels=False, mirror=True, range=ylimits),
    )
    if WRITE_OUTPUT:
        # create the file for donwnload
        img_dir = os.path.join(APPFOLDER, "images/")
        if not os.path.exists(img_dir):
            Path(img_dir).mkdir(parents=True)

        fig.write_image(os.path.join(
            img_dir, f"{username}_PCA_Plot.svg"))
        # create the download CSV and its link
        data_dir = os.path.join(APPFOLDER, "csv/")
        if not os.path.exists(data_dir):
            Path(data_dir).mkdir(parents=True)
        pca_panda.to_csv(os.path.join(
            data_dir, f"{username}_PCA.csv"), index=False)
        CSV_link = f"/files/{url_base}/csv/" \
            f"{username}_PCA.csv"

        # download SVG link
        SVG_link = f"/files/{url_base}/images/" \
            f"{username}_PCA_Plot.svg"

    return fig, CSV_link, SVG_link

In [128]:
####All the functions and constants imports above############################
### All the following sections are for configuration and plotting############

In [129]:
"""_This section read in data from the data management system or from the input
files.
Method 1 use process queue ID, serve address, account and password to read
from data manage process queue_
How to use: set process_queue_id and server_address, username and password
"""

process_queue_id =7272 # example MM PD3 6983, FP 7272
server_address = "10.37.240.41"
username = "XiaofengXie" #user name
password = "" # DO NOT LEAVE your password when uploading to github
if password != "" or process_queue_id == None:
    queue_info, processor_info = queue_info_api(process_queue_id,
                                                    server_address,
                                                    username,
                                                    password)
    data_obj = read_file(queue_info=queue_info, processor_info=processor_info)
else:
    queue_info, processor_info = None, None


# Method 2, use input file for customized tasks
# read  custom data
"""
#FragePipe data
data_obj = read_file(process_app = "FragPipe",
    input1 = "input/Fragpipe/combined_protein.tsv",
    input2 = "input/Fragpipe/combined_peptide.tsv",)
data_obj = read_file(process_app = "DIANN",
                     input1 = "input/DIA/diann-output.pg_matrix.tsv",
                     input2 = "input/DIA/diann-output.pr_matrix.tsv",
                     input3 = "input/DIA/protein.tsv",
                     input4 = "input/DIA/peptide.tsv",
                     input5 = "input/DIA/filelist_diann.txt",)
#ximena pd3 data
data_obj = read_file(process_app = "PD",
                     input1 = "input\PD\With10ngLibraries_Proteins.txt",
                     input2 = "input\PD\With10ngLibraries_PeptideGroups.txt",
                     input3 = "input/DIA/protein.tsv",
                     input4 = "input/DIA/peptide.tsv",
                     input5 = "input\PD\With10ngLibraries_InputFiles.txt",)

#MM PD3 data
data_obj = read_file(process_app = "PD",
                     input1 = "input\MM_PD3\Full_MM_PD3_Proteins.txt",
                     input2 = "input\MM_PD3\Full_MM_PD3_PeptideGroups.txt",
                     input5 = "input\MM_PD3\Full_MM_PD3_InputFiles.txt",)
#pd.set_option('display.max_rows', None)
"""
#display(data_obj)


'\n#FragePipe data\ndata_obj = read_file(process_app = "FragPipe",\n    input1 = "input/Fragpipe/combined_protein.tsv",\n    input2 = "input/Fragpipe/combined_peptide.tsv",)\ndata_obj = read_file(process_app = "DIANN",\n                     input1 = "input/DIA/diann-output.pg_matrix.tsv",\n                     input2 = "input/DIA/diann-output.pr_matrix.tsv",\n                     input3 = "input/DIA/protein.tsv",\n                     input4 = "input/DIA/peptide.tsv",\n                     input5 = "input/DIA/filelist_diann.txt",)\n#ximena pd3 data\ndata_obj = read_file(process_app = "PD",\n                     input1 = "input\\PD\\With10ngLibraries_Proteins.txt",\n                     input2 = "input\\PD\\With10ngLibraries_PeptideGroups.txt",\n                     input3 = "input/DIA/protein.tsv",\n                     input4 = "input/DIA/peptide.tsv",\n                     input5 = "input\\PD\\With10ngLibraries_InputFiles.txt",)\n\n#MM PD3 data\ndata_obj = read_file(process_app = "PD

In [130]:
filelist = [{"input1":"SingleCell_30m_MBR_KO_2_Proteins.txt",
             "input2":"SingleCell_30m_MBR_KO_2_PeptideGroups.txt",
             "input3":"SingleCell_30m_MBR_KO_2_PeptideGroups.txt",
             "input4":"SingleCell_30m_MBR_KO_2_PeptideGroups.txt",
             "input5":"SingleCell_30m_MBR_KO_2_InputFiles.txt",
             "process_app": "PD"},
             {"input1":"SingleCell_30m_MBR_CTR_Proteins.txt",
             "input2":"SingleCell_30m_MBR_CTR_PeptideGroups.txt",
             "input3":"SingleCell_30m_MBR_CTR_Proteins.txt",
             "input4":"SingleCell_30m_MBR_CTR_PeptideGroups.txt",
             "input5":"SingleCell_30m_MBR_CTR_InputFiles.txt",
             "process_app": "PD"}
            ]

x = read_files(grouped_input_files=filelist)
data_obj = outer_join_data_objects(x)
#display(x[1]["protein_abundance"])
#display(data_obj["protein_abundance"])


C:\Users\keiwe\AppData\Local\Temp\ipykernel_11412\2181568643.py:249: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\keiwe\AppData\Local\Temp\ipykernel_11412\2181568643.py:250: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\keiwe\AppData\Local\Temp\ipykernel_11412\2181568643.py:260: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

0       MS2
1       MS2
2       MS2
3       MS2
4       MS2
       ... 
5358    NaN
5359    NaN
5360    NaN
5361    NaN
5362    MS2
Name: A0, Length: 5348, dtype: object
0       MS2
1       MS2
2       MS2
3       MS2
4       MS2
       ... 
5358    NaN
5359    NaN
5360    NaN
5361    NaN
5362    NaN
Name: A1, Length: 5348, dtype: object
0       MS2
1       MS2
2       MS2
3       MS2
4       MS2
       ... 
5358    NaN
5359    NaN
5360    NaN
5361    MS2
5362    NaN
Name: A2, Length: 5348, dtype: object
0       MS2
1       MS2
2       MS2
3       MS2
4       MS2
       ... 
5358    NaN
5359    NaN
5360    NaN
5361    NaN
5362    NaN
Name: A3, Length: 5348, dtype: object
0       MS2
1       MS2
2       MS2
3       MS2
4       MS2
       ... 
5358    NaN
5359    NaN
5360    NaN
5361    NaN
5362    NaN
Name: A4, Length: 5348, dtype: object
0       MS2
1       MS2
2       MS2
3       MS2
4       MS2
       ... 
5358    NaN
5359    NaN
5360    NaN
5361    NaN
5362    NaN
Name: A5, Length: 

C:\Users\keiwe\AppData\Local\Temp\ipykernel_11412\2181568643.py:249: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\keiwe\AppData\Local\Temp\ipykernel_11412\2181568643.py:250: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\keiwe\AppData\Local\Temp\ipykernel_11412\2181568643.py:260: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

0       MS2
1       MS2
2       MS2
3       MS2
4       MS2
       ... 
5342    NaN
5343    NaN
5344    NaN
5345    NaN
5346    NaN
Name: B0, Length: 5333, dtype: object
0       MS2
1       MS2
2       MS2
3       MS2
4       MS2
       ... 
5342    NaN
5343    MBR
5344    NaN
5345    NaN
5346    NaN
Name: B1, Length: 5333, dtype: object
0       MBR
1       MS2
2       MS2
3       MS2
4       MS2
       ... 
5342    NaN
5343    NaN
5344    NaN
5345    NaN
5346    NaN
Name: B2, Length: 5333, dtype: object
0       MS2
1       MS2
2       MS2
3       MS2
4       MS2
       ... 
5342    NaN
5343    MBR
5344    NaN
5345    NaN
5346    NaN
Name: B3, Length: 5333, dtype: object
0       MS2
1       MS2
2       MS2
3       MS2
4       MS2
       ... 
5342    NaN
5343    MBR
5344    NaN
5345    NaN
5346    NaN
Name: B4, Length: 5333, dtype: object
0       MS2
1       MS2
2       MS2
3       MS2
4       MS2
       ... 
5342    NaN
5343    NaN
5344    NaN
5345    NaN
5346    NaN
Name: B5, Length: 

In [142]:
# define group names and assign analysis to different groups(web app version was done through GUI so
# steps are taken to make sure they have same output)
saved_settings = {    
    "group1_name": "Knock Out",
    "group2_name": "Control",
    "group3_name": "",
    "group4_name": "",
    "group5_name": "",
    "group6_name": "",
    "group1_record":[],
    "group2_record":[],
    "group3_record":[],
    "group4_record":[],
    "group5_record":[],
    "group6_record":[],
    
}
filter_in = {
    "group1_record":"KO@A",
    "group2_record":"Ctr@B",
    "group3_record":"",
    "group4_record":"",
    "group5_record":"",
    "group6_record":"",
}

filter_out = {
    "RB_chip2_A11",
    "24_V_chip1_D9",
    "24_T_chip1_D1_ch",
    "48_T_chip2_D1",
    "48_T_chip2_A1",
    "48_T_chip2_C3",
    "72_V_chip1_D9",
    "72_T_chip1_D5",
} #any item in filter_out if contains in the run name will not be used.

#print(data_obj["peptide_ID_Summary"])
for n in range(len(Group_LIST_NAME)):
    if saved_settings[GROUP_NAME[n]] != "" and filter_in[Group_LIST_NAME[n]] != "": #both name and in filter in exist
        i = 0
        if len(str.split(filter_in[Group_LIST_NAME[n]],sep = "@")) == 2:
            user_list = []
            for each_fileID in str.split(filter_in[Group_LIST_NAME[n]],sep = "@")[1:]:
                print(filter)
                for eachIdentifier in data_obj["run_metadata"]["Run Identifier"]:    
                    currentRun = data_obj["run_metadata"][data_obj["run_metadata"]["Run Identifier"] == eachIdentifier]["Run Names"] 
                    if currentRun.size == 1:
                        if each_fileID in eachIdentifier and list(currentRun)[0] not in user_list:
                            user_list.append(list(currentRun)[0])
                    else:
                        print(currentRun)
            print(filter_in[Group_LIST_NAME[n]])
            for run_name in user_list:
                if str.split(filter_in[Group_LIST_NAME[n]],sep = "@")[0] in run_name and (not any(item in run_name for item in filter_out)):
                    saved_settings[Group_LIST_NAME[n]].append({"option":list(data_obj["run_metadata"][data_obj["run_metadata"]["Run Names"] == run_name]["Run Identifier"])[0]})  
                i = i + 1  
        elif len(str.split(filter_in[Group_LIST_NAME[n]],sep = "@")) == 1:
            print("all files")
            for run_name in data_obj["run_metadata"]["Run Names"]:
                if filter_in[Group_LIST_NAME[n]] in run_name and (not any(item in run_name for item in filter_out)):
                    saved_settings[Group_LIST_NAME[n]].append({"option":data_obj["run_metadata"]["Run Identifier"][i]})  
                i = i + 1   
        else:
            print("invalid filter_in format")  
print(saved_settings)

<class 'filter'>
KO@A
<class 'filter'>
Ctr@B
{'group1_name': 'Knock Out', 'group2_name': 'Control', 'group3_name': '', 'group4_name': '', 'group5_name': '', 'group6_name': '', 'group1_record': [{'option': 0    A0
Name: Run Identifier, dtype: object}, {'option': 1    A1
Name: Run Identifier, dtype: object}, {'option': 2    A2
Name: Run Identifier, dtype: object}, {'option': 3    A3
Name: Run Identifier, dtype: object}, {'option': 4    A4
Name: Run Identifier, dtype: object}, {'option': 5    A5
Name: Run Identifier, dtype: object}, {'option': 6    A6
Name: Run Identifier, dtype: object}, {'option': 7    A7
Name: Run Identifier, dtype: object}, {'option': 8    A8
Name: Run Identifier, dtype: object}, {'option': 9    A9
Name: Run Identifier, dtype: object}, {'option': 10    A10
Name: Run Identifier, dtype: object}, {'option': 11    A11
Name: Run Identifier, dtype: object}, {'option': 12    A12
Name: Run Identifier, dtype: object}, {'option': 13    A13
Name: Run Identifier, dtype: object}, 

In [132]:
#ID plot(protein and peptides)
plot_options={
            "mean label": "True",
            "error bar": "stdev",
            "X Title": "Conditions",
            "Y Title": "Protein Identification",
            "color": ["blue", "red", "black", "yellow", "green", "purple",
                      "orange", "brown", "pink", "gray", "olive", "cyan"],
            "width": 700,
            "height": 450,
            "font": dict(size=16, family="Arial black"),
            "ID mode": "total",
            "help for information only": \
            "Mean label options: True or False." \
            "error bar options: stdev or ci95." \
            "color: the first few colors will be used"
            "ID mode options: total, MS2, stacked." \
        }
plot_options["plot_type"] = "1" # 1 is protein, 2 is peptide
figure ,_ ,_ =ID_plots(data_obj, plot_options, saved_settings)
figure.show()
#figure.write_image("images/test.svg")


In [133]:
# CV violin plot
plot_options={    
        "mean label": "True",
        "box": "True",
        "X Title": "Conditions",
        "Y Title": "CV of Abundance (%)",
        "color": ["blue", "red", "black", "yellow", "green", "purple",
                  "orange", "brown", "pink", "gray", "olive", "cyan"],
        "width": 700,
        "height": 450,
        "font": dict(size=16, family="Arial black"),
        "violinmode": "overlay",
        "ylimits": [0, 100],
        "help for information only": \
        "Mean label options: True or False." \
        "color: the first few colors will be used"\
        "violinmode: group or overlay." \
    }
figure, _, _ =CV_plots(data_obj, plot_options, saved_settings)
figure.show()
#figure.write_image("images/test.svg")


In [134]:
# ID Venns plot
plot_options={
            "compare groups": [0,1],
            "title": "Venn Diagram",
            "opacity": 0.75,
            "color": ["#00FF00", "#FFFF00", "#FF0000", "yellow", "red",
                      "green", "purple", "orange", "brown", "pink",
                      "gray",  "olive", "cyan", "blue",  "black", ],

            "help for information only": \
            "color: the first few colors will be used"
        }
figure, _, _ =venns_plots(data_obj, plot_options, saved_settings)
figure.show()
#figure.write_image("images/test.svg")



In [135]:
# Abundance Volcano plot
plot_options={
            "compare groups": [0, 1],
            "title": "Volcano Plot",
            "X Title": "Log2 Fold Change",
            "Y Title": "-Log10(p-value)",
            "up color": "red",
            "down color": "blue",
            "all color": "gray",
            "width": 700,
            "height": 450,
            "font": dict(size=16, family="Arial black"),
            "ylimits": [],
            "xlimits": [],
            "help for information only": \
            "compare groups: list of two numbers default [0,1] means compare" \
            "the second group again the first group." \
            "xlimits: list or tuple of two numbers [0,10]." \
        }
figure, _, _ =volcano_plots(data_obj, plot_options, saved_settings)
figure.show()
#figure.write_image("images/test.svg")

C:\Users\keiwe\AppData\Local\Temp\ipykernel_11412\3277781000.py:93: FutureWarning:

The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [136]:
# Abundance PCA plot
plot_options={

            "compare groups": [0,1],
            "title": "PCA Analysis",
            "color": ["blue", "red", "black", "yellow", "green", "purple",
                      "orange", "brown", "pink", "gray", "olive", "cyan"],
            "symbol": ['star', 'circle'],
            "marker_size": 8,
            "width": 700,
            "height": 450,
            "font": dict(size=16, family="Arial black"),
            "ylimits": [],
            "xlimits": [],
            "help for information only": \
            "compare groups: list of two numbers default [0,1] means compare" \
            "color: the first few colors will be used"
            "xlimits: list or tuple of two numbers [0,10]." \
        }
figure, _, _ =PCA_plots(data_obj, plot_options, saved_settings)
figure.show()
#figure.write_image("images/test.svg")

{'run_metadata': [], 'protein_abundance':      Accession         A0         A1         A2         A3         A4  \
0       Q15149  20.204807  20.317260  20.083906  19.093529  19.835975   
1       P21333  22.709420  21.807064  21.217696  22.833179  21.773056   
2       Q09666  21.635035  21.204353  20.060117  21.364701  20.984860   
3       P78527  21.192377  21.591707  21.841582  21.068285  21.677078   
4       Q14204  20.655793  20.814266  20.707763  20.863151  20.679751   
...        ...        ...        ...        ...        ...        ...   
3011    Q13641  12.145291  11.927559  12.245877  11.654339        NaN   
3012    Q32MK0  14.736426  14.689665  14.845846  14.794046  14.449774   
3013    Q92839        NaN        NaN        NaN        NaN        NaN   
3014    Q9H081        NaN        NaN        NaN        NaN        NaN   
3015    Q9H008  15.067056        NaN  15.203346        NaN  15.421204   

             A5         A6         A7         A8  ...        A14        A15  \
0 